In [11]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import sklearn
import pandas as pd
import scipy.stats as stats

#from keras import Sequential
#from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [2]:
neuron_data = {}

files = sorted(os.listdir('../neuron_logs/train_data'))


In [3]:
def reduce_to_statistics(activations, labels, debug=False):
    sorted_data = []
    for i in range(10): #hardcoded MOFO
        sorted_data += [[]]
    for i, a in zip(labels, activations):
        sorted_data[i] += [a]
    if debug:
        return(sorted_data)
    statistics = []
    for ar in sorted_data:
        curr_stats = stats.describe(ar)
        statistics += [curr_stats.mean, curr_stats.variance, curr_stats.skewness, curr_stats.kurtosis, curr_stats.minmax[0],
                       curr_stats.minmax[1], curr_stats.nobs]
        #print(statistics)
    return(statistics)

In [4]:
def extract_data(filename, fin = 10, activations_no = 1000, target = 'usefulness_loss', shuffle = True):
    
    features, labels = [], []
    with open(os.path.join('..', 'neuron_logs', 'train_data', filename), 'r') as f:
        neuron_data = json.load(f)
        for e in neuron_data.keys():
            if e == '0' or int(e) > fin:
                continue
            for neuron in neuron_data[e]:
                if ' ' not in neuron:
                    continue
                current_data = neuron_data[e][neuron]
                important_features = []
                important_features = current_data['activations'][:activations_no]
                important_features += reduce_to_statistics(current_data['activations'], neuron_data[e]['original_labels'])
                important_features += [current_data['depth']]
                important_features += [current_data['inverse_depth']]
                important_features += [current_data['width']]
                # important_features += [current_data['input_weights']]
                # important_features += [current_data['output_weights']]
                important_features += [current_data['reg_loss_in_layer']]
                important_features += [e]
                features += [important_features]
                #labels += [current_data[target]]
                labels += [current_data[target]]
    
    if shuffle:
        shuffled = sklearn.utils.shuffle(np.concatenate([np.array(features), np.array(labels).reshape(-1, 1)], axis=1))
        features, labels = shuffled[:, :-1], shuffled[:, -1]
    
    return(np.array(features, dtype=np.float32), np.array(labels, dtype=np.float32))


In [5]:
from sklearn.datasets import make_classification
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim


import skorch
from skorch import NeuralNetClassifier
from skorch import NeuralNetRegressor

In [6]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=1000,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(X_train.shape[1], num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, num_units//2)
        self.dense2 = nn.Linear(num_units//2, num_units//2)
        self.dense3 = nn.Linear(num_units//2, num_units//2)
        self.denselast = nn.Linear(num_units//2, 100)
        self.output = nn.Linear(100, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = F.relu(self.dense2(X))
        X = F.relu(self.dense3(X))
        X = F.relu(self.denselast(X))
        X = self.output(X)
        return X

In [7]:
from skorch.callbacks import LRScheduler

lrscheduler = LRScheduler(monitor='train_loss',
            policy='ReduceLROnPlateau', # step_size=15, gamma=0.5)
            mode='min', factor=0.2, patience=10, verbose=True, 
            threshold=0.00001, threshold_mode='rel', 
            cooldown=0, min_lr=0, eps=1e-08)

from skorch.callbacks import Checkpoint

checkpoint = Checkpoint(
    f_params='best_model.pt', monitor='valid_loss_best')

from skorch.callbacks import EarlyStopping

earlystopping = EarlyStopping(
    monitor='train_loss',
    patience=20,
    threshold=0.0001,
    threshold_mode='abs',
    lower_is_better=True,)

In [14]:
net = NeuralNetRegressor(
    RegressorModule,
    max_epochs=10,
    lr=0.1,
    optimizer=optim.SGD,
    optimizer__momentum=0.9,
    # callbacks=[skorch.callbacks.LRScheduler(skorch.callbacks.WarmRestartLR)],
    callbacks = [lrscheduler, checkpoint, earlystopping],
    warm_start=True
)

In [16]:
X_test, y_test = extract_data(files[0])
scaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)
valids, tests = [], []

In [17]:
# train
for epoch in range(5):
    for i, filename in enumerate(files[1:]):
        print(f'Opening file {filename}')
        features, labels = extract_data(filename)
        scaler = StandardScaler()
        scaler.partial_fit(features)
        features = scaler.transform(features)

        X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.1, random_state=0)

        net.partial_fit(X_train, y_train.reshape(-1, 1))

        y_pred = net.predict(X_valid)
        valids += [mean_squared_error(y_pred, y_valid)]
        print(f'MSE on valid: {mean_squared_error(y_pred, y_valid)}')
        y_pred = net.predict(X_test)
        tests += [mean_squared_error(y_pred, y_test)]
        print(f'MSE on unknown network: {mean_squared_error(y_pred, y_test)}')

Opening file output_20191015-203714.json
     11        0.0165        0.0186        0.2507
     12        0.0164        0.0178        0.2917
     13        0.0153        0.0170        0.2489
     14        0.0145        0.0167        0.2455
     15        0.0142        0.0165        0.2489
     16        0.0136        0.0158        0.2575
     17        0.0127        0.0152        0.2571
     18        0.0119        0.0148        0.2462
     19        0.0111        0.0144        0.2611
     20        0.0104        0.0137        0.2569
MSE on valid: 0.012700558640062809
MSE on unknown network: 0.027011197060346603
Opening file output_20191015-215448.json
     21        0.0243        0.0243        0.2518
     22        0.0214        0.0231        0.2478
     23        0.0182        0.0192        0.2430
     24        0.0142        0.0138        0.2494
     25        0.0110        0.0102     +  0.2483
     26        0.0093        0.0071     +  0.2465
     27        0.0080        0.0068   

KeyboardInterrupt: 